### DEPENDENCIES

In [19]:
%pip install datasets
%pip install pytorch_lightning==1.6.0
%pip install transformers
%pip install wandb
%pip install pyyaml
%pip install optuna==2.0.0
%pip install streamlit
%pip install protobuf
%pip install pyngrok
%pip install pandas
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HADOOP\Desktop\NLP\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HADOOP\Desktop\NLP\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\HADOOP\Desktop\NLP\venv\Scripts\python.exe -m pip install --upgrade pip' command.



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HADOOP\Desktop\NLP\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HADOOP\Desktop\NLP\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name()

c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Quadro RTX 8000'

### DATA ACQUISITION

##### VALIDATION_DATASET

In [3]:
import os
from typing import Optional
from datasets import load_dataset, IterableDataset
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling

# current working directory changes when imported from other modules, so to ensure para_nmt_path is correct we store
# the absolute path to the module for reference.
package_directory = os.path.dirname(os.path.abspath('__file__'))


class VALIDATION_DATASET(LightningDataModule):
    
    file_path = r"C:\Users\HADOOP\Desktop\NLP\validation.txt"

    def __init__(self, opt_type, batch_size, steps_per_epoch, num_workers=0, seed=69, pre_tokenize=True):
        """
        Parameters
        ----------
        opt_name: str
            name of the OPT model type (i.e. facebook/opt-350m)
        batch_size: int
            batch_size output by dataloader
        steps_per_epoch: int
            dataset_size = steps_per_epoch * batch_size
            Since we do not know the dataset size we simply leave it to the user to determine how many steps per epoch
            we should have.
        num_workers: int
            refer to note above on PR https://github.com/huggingface/datasets/pull/4375
        seed: int
            haha funny number
        pre_tokenize: bool
            should we tokenize the texts (if true: dataset will return tokenized ids instead of source text)
        """

        super().__init__()
        self.opt_type = opt_type
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch
        self.num_workers = num_workers
        self.seed = seed
        self.pre_tokenize = pre_tokenize

        # init None to make pycharm happy
        self.tokenizer = None
        self.dataset = None

    def prepare_data(self) -> None:
        # download and cache
        GPT2Tokenizer.from_pretrained(self.opt_type)

    def setup(self, stage: Optional[str] = None) -> None:
        # load tokenizer (should be cached)
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.opt_type, use_fast=False)

        # preprocess function for the dataset's entries
        def preprocess(examples):
            # list of len batch
            batch = examples['text']
            processed_batch = list()
            for i in batch:
                # replace the \t splitting with a '</s>' token to denote source-target
                processed_batch.append(str.replace(i, "\t", self.tokenizer.eos_token))

            if self.pre_tokenize:
                outputs = self.tokenizer(
                    processed_batch,
                    truncation=True,
                    max_length=69,
                )
            else:
                outputs = {"source": processed_batch}
            return outputs

        # init dataset in streaming mode
        self.dataset = load_dataset("text", data_files=self.file_path, streaming=True)['train']
        
        # elements within buffer size will be shuffled as they are loaded in
        self.dataset = self.dataset.shuffle(seed=self.seed, buffer_size=10_000)
        
        # preprocessing will take place while being streamed by dataloader
        self.dataset = self.dataset.map(preprocess, batched=True, remove_columns=['text'])
        
        # ensure pytorch tensors are returned
        self.dataset = self.dataset.with_format("torch")

        # monkeypatch of __len__ function in the dataloader so that the trainer knows how many
        # steps there are per epoch. Sure this violates many programming paradigms but it works.
        n = self.steps_per_epoch

        def __len__(self):
            return n

        IterableDataset.__len__ = __len__

    # dataloaders are basically all the same since we cannot split a streamed dataset
    def train_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def val_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def test_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def predict_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader


if __name__ == "__main__":
    model_name = "facebook/opt-1.3b"
    datamodule = VALIDATION_DATASET(model_name, 1, 1000, seed=1337)
    datamodule.setup()
    dl = datamodule.val_dataloader()
    it = iter(dl)

    for i in range(10):
        print(datamodule.tokenizer.batch_decode(next(it)['input_ids'])[0])

Using custom data configuration default-b06b12bf792cbc7d


</s>can it restore heat to a corpse? ''</s>can he return the heat to the corpse? ``
</s>`` how about 'yes '? '' onrad asked softly.</s>'how about plain, yes '? 'onrad asked quietly.
</s>wide at one end and narrow at the other, its shape reminded harad of the stringed instruments musicians played on feast days.</s>at one end was wide, on the other narrow, and its shape reminded him of a strident instrument that was played by musicians.
</s>it involved a squad of imperial junior scouts, a flag ceremony, a wooden footbridge, and my cousin ivan.</s>he was involved in that section of the imperial young scouts, a wooden bridge, and my cousin ivan.
</s>lived in a hut, even took vows, or whatever women do in such countries.</s>she lived in a shack, made a promise, or what women do in those countries.
</s>the privacy level of the message</s>level of confidential message
</s>`` no problem, '' the guard said, heaving the gate open.</s>`` there are no problems, '' he said, opening the gate.
</s>i 

##### PARANMT_50M_DATASET

https://paperswithcode.com/dataset/paranmt-50m



In [2]:
import os
from typing import Optional
from datasets import load_dataset, IterableDataset
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling

# current working directory changes when imported from other modules, so to ensure para_nmt_path is correct we store
# the absolute path to the module for reference.
package_directory = os.path.dirname(os.path.abspath('__file__'))


class PARANMT_50M_DATASET(LightningDataModule):
    
    file_path = os.path.join(package_directory, "para-nmt-5m-processed.zip")

    def __init__(self, opt_type, batch_size, steps_per_epoch, num_workers=0, seed=69, pre_tokenize=True):
        """
        Parameters
        ----------
        opt_name: str
            name of the OPT model type (i.e. facebook/opt-350m)
        batch_size: int
            batch_size output by dataloader
        steps_per_epoch: int
            dataset_size = steps_per_epoch * batch_size
            Since we do not know the dataset size we simply leave it to the user to determine how many steps per epoch
            we should have.
        num_workers: int
            refer to note above on PR https://github.com/huggingface/datasets/pull/4375
        seed: int
            haha funny number
        pre_tokenize: bool
            should we tokenize the texts (if true: dataset will return tokenized ids instead of source text)
        """

        super().__init__()
        self.opt_type = opt_type
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch
        self.num_workers = num_workers
        self.seed = seed
        self.pre_tokenize = pre_tokenize

        # init None to make pycharm happy
        self.tokenizer = None
        self.dataset = None

    def prepare_data(self) -> None:
        # download and cache
        GPT2Tokenizer.from_pretrained(self.opt_type)

    def setup(self, stage: Optional[str] = None) -> None:
        # load tokenizer (should be cached)
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.opt_type, use_fast=False)

        # preprocess function for the dataset's entries
        def preprocess(examples):
            # list of len batch
            batch = examples['text']
            processed_batch = list()
            for i in batch:
                # replace the \t splitting with a '</s>' token to denote source-target
                processed_batch.append(str.replace(i, "\t", self.tokenizer.eos_token))

            if self.pre_tokenize:
                outputs = self.tokenizer(
                    processed_batch,
                    truncation=True,
                    max_length=69,
                )
            else:
                outputs = {"source": processed_batch}
            return outputs

        # init dataset in streaming mode
        self.dataset = load_dataset("text", data_files=self.file_path, streaming=True)['train']
        
        # elements within buffer size will be shuffled as they are loaded in
        self.dataset = self.dataset.shuffle(seed=self.seed, buffer_size=10_000)
        
        # preprocessing will take place while being streamed by dataloader
        self.dataset = self.dataset.map(preprocess, batched=True, remove_columns=['text'])
        
        # ensure pytorch tensors are returned
        self.dataset = self.dataset.with_format("torch")

        # monkeypatch of __len__ function in the dataloader so that the trainer knows how many
        # steps there are per epoch. Sure this violates many programming paradigms but it works.
        n = self.steps_per_epoch

        def __len__(self):
            return n

        IterableDataset.__len__ = __len__

    # dataloaders are basically all the same since we cannot split a streamed dataset
    def train_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def val_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def test_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def predict_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader


if __name__ == "__main__":
    model_name = "facebook/opt-1.3b"
    datamodule = PARANMT_50M_DATASET(model_name, 1, 1000, seed=1337)
    datamodule.setup()
    dl = datamodule.val_dataloader()
    it = iter(dl)

    for i in range(10):
        print(datamodule.tokenizer.batch_decode(next(it)['input_ids'])[0])

Using custom data configuration default-12387731b9302b2b


</s>where a reference is made to this paragraph, article 5 of regulation ( eu ) no 182/2011 shall apply.</s>where reference is made to this article, article 5 of regulation ( eu ) no 182 / 2011 shall apply.
</s>don't worry, pete, i'm coming to save you.</s>don't worry, pete.
</s>opening of sitting the sitting opened at 09.00. 2.</s>start of the session the session started at 9 p.m.
</s>it'il get as fat as you are.</s>he 'll be as fat as you.
</s>that had struck jess and will as funny, because everyone knew that your posterior was the scientific name for your situpon.</s>this seemed to be jess and will's funny, since everyone knew that the back tie was a scientific name for the butt.
</s>his long, lethal fingers rhythmically clawed the ground as they gained strength.</s>the long, deadly fingers moved rhythmically into the soil, and the original forces were slow.
</s>oh... honey.</s>oh, baby...
</s>i couldn't tell half the time if he was talking... or you were reading his mind.</s>i coul

##### PARABANK_Dataset
https://paperswithcode.com/dataset/parabank

In [3]:
"""
README from parabank-2.0.zip
The TSV file contains ParaBank 2, a diverse collection of paraphrases generated
through bilingual generation. Details of the dataset and how it's created can
be found here:
Hu, J. E., A. Singh, N. Holzenberger, M. Post, & B. Van Durme. 2019. Large-scale,
Diverse, Paraphrastic Bitexts via Sampling and Clustering. Proceedings of CoNLL 2019,
Hong Kong, Nov 3 – Nov 4, 2019.
Each line of the file contains a bilingual dual-condition score, a reference
sentence, and paraphrases of the same reference sentence. A reference sentence may
have between one to five distinct paraphrases. The lines are in descending
order of the dual-conditioned score, a measurement of the quality of the
original bilingual sentence pair. Within the same line, paraphrases are ranked by
model score as described in the paper - i.e., the first paraphrase from left
to right correspond to the system with subscript "1" in evaluation, and the
last to "5". All sentences are raw text (untokenized). The reference sentences
appear in ascending order of their bidirectional model scores (the lower the
better), which we use to filter the bilingual resource used to generate ParaBank 2.
"""
from typing import Optional
from datasets import load_dataset, IterableDataset
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling


class PARABANK_Dataset(LightningDataModule):
    """
    LightningDataModule for parabank dataset for causal language modelling
    Note on num_workers: https://github.com/huggingface/datasets/pull/4375
    IterableDatasets do not support Dataloaders with num_workers > 0. Watch the PR to see if the fix will be merged.
    """
    parabank_url = "http://cs.jhu.edu/~vandurme/data/parabank-2.0.zip"

    def __init__(self, opt_name, batch_size, steps_per_epoch, num_workers=0, seed=69, pre_tokenize=True):
        """
        Parameters
        ----------
        opt_name: str
            name of the OPT model type (i.e. facebook/opt-350m)
        batch_size: int
            batch_size output by dataloader
        steps_per_epoch: int
            dataset_size = steps_per_epoch * batch_size
            Since we do not know the dataset size we simply leave it to the user to determine how many steps per epoch
            we should have.
        num_workers: int
            refer to note above on PR https://github.com/huggingface/datasets/pull/4375
        seed: int
            haha funny number
        pre_tokenize: bool
            should we tokenize the texts (if true: dataset will return tokenized ids instead of source text)
        """

        super().__init__()
        self.opt_name = opt_name
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch
        self.num_workers = num_workers
        self.seed = seed
        self.pre_tokenize = pre_tokenize

        # init None to make pycharm happy
        self.tokenizer = None
        self.dataset = None

    def prepare_data(self) -> None:
        # download and cache
        GPT2Tokenizer.from_pretrained(self.opt_name)

    def setup(self, stage: Optional[str] = None) -> None:
        # load tokenizer (should be cached)
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.opt_name, use_fast=False)

        # preprocess function for the dataset's entries
        def preprocess(examples):
            # list of len batch
            batch = examples['text']
            processed_batch = list()
            for i in batch:
                # split by \t (it is a tsv file) and omit the initial dual-condition score (it is useless)
                i = i.split('\t')[1:]
                # filter entries without paraphrases and split them with a '</s>' token to denote source-target
                if len(i) > 1:
                    processed_batch.append(i[0] + self.tokenizer.eos_token + i[1])

            if self.pre_tokenize:
                outputs = self.tokenizer(
                    processed_batch,
                    truncation=True,
                    max_length=69,
                )
            else:
                outputs = {"source": processed_batch}
            return outputs

        # init dataset in streaming mode
        self.dataset = load_dataset("text", data_files=self.parabank_url, streaming=True)['train']
        # elements within buffer size will be shuffled as they are loaded in
        self.dataset = self.dataset.shuffle(seed=self.seed, buffer_size=10_000)
        # preprocessing will take place while being streamed by dataloader
        self.dataset = self.dataset.map(preprocess, batched=True, remove_columns=['text'])
        # ensure pytorch tensors are returned
        self.dataset = self.dataset.with_format("torch")

        # monkeypatch of __len__ function in the dataloader so that the trainer knows how many
        # steps there are per epoch. Sure this violates many programming paradigms but it works.
        n = self.steps_per_epoch

        def __len__(self):
            return n

        IterableDataset.__len__ = __len__

    # dataloaders are basically all the same since we cannot split a streamed dataset
    def train_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def val_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def test_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def predict_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader


if __name__ == "__main__":
    model_name = "facebook/opt-1.3b"
    datamodule = PARABANK_Dataset(model_name, 1, 1000, seed=1337)
    datamodule.setup()
    dl = datamodule.val_dataloader()
    it = iter(dl)

    for i in range(10):
        print(datamodule.tokenizer.batch_decode(next(it)['input_ids'])[0])

Using custom data configuration default-a233796b5026b737


</s>E-3612/10 (RO) Elena Oana Antonescu (PPE) to the Commission (25 May 2010)</s>
</s>Much of the poor majority of the world is mired in a vicious circle of disease, poverty, and political instability.</s>A great deal of the poor majority of the world is drowned in an enchanted ring of ills, poverty, and political instalency.
</s>You promised me 24 hours.</s>I've been promised 24 hours.
</s>Management of deep-sea fish stocks (vote)</s>Management of fishing stocks in deep waters (vote)
</s>Forty seconds!</s>40 seconds!
</s>Regulation (EEC) No 3846/87 should therefore be amended accordingly.</s>Regulation (EEC) No 3846/87 should therefore be amended in accordance with the above-mentioned provisions.
</s>Richard saved me.</s>Richard rescued me.
</s>He had a good teacher.</s>He had a good tutor.
</s>For the first time since 1974.</s>First since 1974.
</s>You are in my hospital.</s>You are in my Hospital.


##### QUORA_Dataset

In [6]:
import os
from typing import Optional
from datasets import load_dataset, IterableDataset
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling

# current working directory changes when imported from other modules, so to ensure para_nmt_path is correct we store
# the absolute path to the module for reference.
package_directory = os.path.dirname(os.path.abspath('__file__'))


class QUORA_DATASET(LightningDataModule):
    
    hugging_dir = "HanHan055/quora_question_answer_pair"
    def __init__(self, opt_type, batch_size, steps_per_epoch, num_workers=0, seed=69, pre_tokenize=True):
        """
        Parameters
        ----------
        opt_name: str
            name of the OPT model type (i.e. facebook/opt-350m)
        batch_size: int
            batch_size output by dataloader
        steps_per_epoch: int
            dataset_size = steps_per_epoch * batch_size
            Since we do not know the dataset size we simply leave it to the user to determine how many steps per epoch
            we should have.
        num_workers: int
            refer to note above on PR https://github.com/huggingface/datasets/pull/4375
        seed: int
            haha funny number
        pre_tokenize: bool
            should we tokenize the texts (if true: dataset will return tokenized ids instead of source text)
        """

        super().__init__()
        self.opt_type = opt_type
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch
        self.num_workers = num_workers
        self.seed = seed
        self.pre_tokenize = pre_tokenize

        # init None to make pycharm happy
        self.tokenizer = None
        self.dataset = None

    def prepare_data(self) -> None:
        # download and cache
        GPT2Tokenizer.from_pretrained(self.opt_type)

    def setup(self, stage: Optional[str] = None) -> None:
        # load tokenizer (should be cached)
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.opt_type, use_fast=False)

        # preprocess function for the dataset's entries
        def preprocess(examples):
            # list of len batch
            processed_batch = list()
            for i,j in zip(examples['source'], examples['target']):
                # replace the \t splitting with a '</s>' token to denote source-target
                processed_batch.append(i+ self.tokenizer.eos_token +j)

            if self.pre_tokenize:
                outputs = self.tokenizer(
                    processed_batch,
                    truncation=True,
                    max_length=69,
                )
            else:
                outputs = {"source": processed_batch}
            return outputs

        # init dataset in streaming mode
        self.dataset = load_dataset("csv","quora.csv", streaming=True)['train']
        
        # elements within buffer size will be shuffled as they are loaded in
        self.dataset = self.dataset.shuffle(seed=self.seed, buffer_size=10_000)
        
        # preprocessing will take place while being streamed by dataloader
        self.dataset = self.dataset.map(preprocess, batched=True, remove_columns=['source','target'])
        
        # ensure pytorch tensors are returned
        self.dataset = self.dataset.with_format("torch")

        # monkeypatch of __len__ function in the dataloader so that the trainer knows how many
        # steps there are per epoch. Sure this violates many programming paradigms but it works.
        n = self.steps_per_epoch

        def __len__(self):
            return n

        IterableDataset.__len__ = __len__

    # dataloaders are basically all the same since we cannot split a streamed dataset
    def train_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def val_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def test_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def predict_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader


if __name__ == "__main__":
    model_name = "facebook/opt-1.3b"
    datamodule = QUORA_DATASET(model_name, 1, 1000, seed=1337)
    datamodule.setup()
    dl = datamodule.val_dataloader()
    it = iter(dl)

    for i in range(10):
        print(datamodule.tokenizer.batch_decode(next(it)['input_ids'])[0])

Resolving data files: 100%|██████████| 80/80 [00:00<00:00, 161241.86it/s]
Using custom data configuration quora.csv-ede1aa58c31497a3
c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\datasets\download\streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
Failed to read file 'C:\Users\HADOOP\Desktop\NLP\venv\Lib\site-packages\pytorch_lightning\plugins\training_type\tpu_spawn.py' with error <class 'pandas.errors.ParserError'>: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\HADOOP\AppData\Local\Temp\ipykernel_15564\184983851.py:133 in <module>                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\HADOOP\\AppData\\Local\\Temp\\ipykernel_15564\\184983851.py'                         │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\torch\utils\data\dataloader.py:628 in         │
│ __next__                                                                                         │
│                                                                                                  │
│    625 │   │   │   if self._sampler_iter is None:                                                │
│    626 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    627 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  628 │   │   │   data = self._next_data()                                                      │
│    629 │   │   │   self._num_yielded += 1                                                        │
│    630 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    631 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\torch\utils\data\dataloader.py:671 in         │
│ _next_data                                                                                       │
│                                                                                                  │
│    668 │                                                                                         │
│    669 │   def _next_data(self):                                                                 │
│    670 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  671 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    672 │   │   if self._pin_memory:                                                              │
│    673 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    674 │   │   return data                                                                       │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\torch\utils\data\_utils\fetch.py:34 in fetch  │
│                                                                                                  │
│   31 │   │   │   data = []                                                                       │
│   32 │   │   │   for _ in possibly_batched_index:                                                │
│   33 │   │   │   │   try:                                                                        │
│ ❱ 34 │   │   │   │   │   data.append(next(self.dataset_iter))                                    │
│   35 │   │   │   │   except StopIteration:                                                       │
│   36 │   │   │   │   │   self.ended = True                                                       │
│   37 │   │   │   │   │   break                                                                   │
│                                                                                                  │
│ c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\datasets\iterable_dataset.py:846 in __iter__  │
│                                                            

##### DataCombiner

In [2]:
from typing import List, Type, Optional

from datasets import IterableDataset, interleave_datasets
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling


class DataCombiner(LightningDataModule):
    """
    LightningDataModule for combining different datasets for causal language modelling
    Note on num_workers: https://github.com/huggingface/datasets/pull/4375
    IterableDatasets do not support Dataloaders with num_workers > 0. Watch the PR to see if the fix will be merged.
    """
    def __init__(self, opt_name, batch_size, steps_per_epoch, datamodules: List[Type[LightningDataModule]],
                 probabilities: List[float], num_workers=0, seed=69, pre_tokenize=True):
        """
        Parameters
        ----------
        opt_name: str
            Name of model type
        batch_size: int
            batch_size output by dataloader
        steps_per_epoch: int
            dataset_size = steps_per_epoch * batch_size
            Since we do not know the dataset size we simply leave it to the user to determine how many steps per epoch
            we should have.
        datamodules: List[Type[LightningDataModule]]
            List specifying the datamodules whose datasets will be interleaved
        probabilities: List[float]
            List of probabilities for respective datamodules that should sum to 1
        num_workers: int
            refer to note above on PR https://github.com/huggingface/datasets/pull/4375
        seed: int
            haha funny number
        pre_tokenize: bool
            should we tokenize the texts (if true: dataset will return tokenized ids instead of source text)
        """
        super().__init__()
        self.opt_name = opt_name
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch
        self.num_workers = num_workers
        self.seed = seed
        self.pre_tokenize = pre_tokenize
        self.datamodules = datamodules
        self.probabilities = probabilities
        self.tokenizer = None
        self.dataset = None

        # sanity check
        assert sum(self.probabilities) == 1, "Probabilities for interleaved datasets do not sum to 1.0"

    def prepare_data(self) -> None:
        # download and cache
        GPT2Tokenizer.from_pretrained(self.opt_name)

    def setup(self, stage: Optional[str] = None) -> None:
        # tokenizer is not actually used once instantiated but to stay consistent with other datamodule implementations
        # we instantiate it anyway
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.opt_name, use_fast=False)

        # instantiate all the datamodules and extract the dataset from them
        datasets = list()
        for datamodule in self.datamodules:
            dm = datamodule(self.opt_name, self.batch_size, self.steps_per_epoch,
                            seed=self.seed, pre_tokenize=self.pre_tokenize)
            dm.setup()
            datasets.append(dm.dataset)

        self.dataset = interleave_datasets(datasets, probabilities=self.probabilities, seed=self.seed)
        self.dataset = self.dataset.with_format("torch")

        # monkeypatch of __len__ function in the dataloader so that the trainer knows how many
        # steps there are per epoch. Sure this violates many programming paradigms but it works.
        n = self.steps_per_epoch

        def __len__(self):
            return n

        IterableDataset.__len__ = __len__

    # dataloaders are basically all the same since we cannot split a streamed dataset
    def train_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def val_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def test_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader

    def predict_dataloader(self):
        dataloader = DataLoader(self.dataset,
                                batch_size=self.batch_size,
                                num_workers=self.num_workers)
        if self.pre_tokenize: dataloader.collate_fn = DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        return dataloader


if __name__ == "__main__":
    model_name = "facebook/opt-1.3b"
    datamodule = DataCombiner(model_name, 1, 100, [PARABANK_Dataset, PARANMT_50M_DATASET],
                                        probabilities=[0.5,0.5], seed=1337, pre_tokenize=False)
    datamodule.setup()
    dl = datamodule.val_dataloader()
    it = iter(dl)

    for i in range(10):
        print(next(it))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\HADOOP\AppData\Local\Temp\ipykernel_21468\1081730767.py:115 in <module>                 │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\HADOOP\\AppData\\Local\\Temp\\ipykernel_21468\\1081730767.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'PARABANK_Dataset' is not defined

### Soft Promp Setup

##### SoftEmbedding

In [5]:
"""
Copied from https://github.com/kipgparker/soft-prompt-tuning/blob/main/soft_embedding.py
It's really that simple. huh.
"""

import torch
import torch.nn as nn


class SoftEmbedding(nn.Module):
    def __init__(self,
                 wte: nn.Embedding,
                 n_tokens: int = 10,
                 random_range: float = 0.5,
                 initialize_from_vocab: bool = True):
        """appends learned embedding to
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                                  n_tokens,
                                                                                  random_range,
                                                                                  initialize_from_vocab))

    def initialize_embedding(self,
                             wte: nn.Embedding,
                             n_tokens: int = 10,
                             random_range: float = 0.5,
                             initialize_from_vocab: bool = True) -> torch.Tensor:
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)

    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specific embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

##### SoftOPTModelWrapper

In [6]:
import re
from functools import reduce
from typing import Dict

from pytorch_lightning import LightningModule, Callback
from torch.optim import Adam, SGD, Optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, _LRScheduler
from transformers.models.opt.modeling_opt import *

import wandb
import os


class SoftOPTModelWrapper(OPTForCausalLM):
    """Wrapper class for OPTForCausalLM to add learnable embedding functionality
    Simply initialise it with from_pretrained OPT files and it should work out of the box.
    """
    _keys_to_ignore_on_load_missing = [r"soft_embedding.wte.weight", r"soft_embedding.learned_embedding",
                                       r"lm_head.weight"]

    def __init__(self, config: OPTConfig):
        super().__init__(config)

        # init parameters for embedding
        self.n_tokens = wandb.config["embedding_n_tokens"]
        self.init_from_vocab = wandb.config["init_from_vocab"]

        # initialise the embedding to learn
        self.soft_embedding = SoftEmbedding(self.get_input_embeddings(),
                                            n_tokens=self.n_tokens,
                                            initialize_from_vocab=self.init_from_vocab)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *model_args, **kwargs):
        """Incredibly scuffed but we have to set the input embeddings to the soft embeddings only AFTER
        the pretrained weights have been loaded in. All parameters are the same as a normal from_pretrained() call
        """

        pretrained_model = super().from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)
        pretrained_model.set_input_embeddings(pretrained_model.soft_embedding)
        return pretrained_model

    def forward(self,
                input_ids: torch.LongTensor = None,
                attention_mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.LongTensor] = None,
                **kwargs):
        """Shitty forward pass
        need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
        even though it does not matter what we pad input_ids with, it's just to make HF happy
        """

        batch_size = input_ids.shape[0]
        # Note: concatenation of tensors have to happen on the same device
        # concat padding representing our learned embedding tokens for batched inputs
        # inputs come in as (batch_size, seq_len) and are padded to be (batch_size, n_tokens + seq_len)
        input_ids = torch.cat([torch.full((batch_size, self.n_tokens), 50256).to(input_ids.device), input_ids], dim=1)
        attention_mask = torch.cat(
            [torch.full((batch_size, self.n_tokens), 1).to(attention_mask.device), attention_mask], dim=1)
        if labels is not None:
            labels = torch.cat([torch.full((batch_size, self.n_tokens), 50256).to(labels.device), labels], dim=1)

        return super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)


class ParaphraseOPT(LightningModule):
    def __init__(self, model_name="facebook/opt-350m", init_optimizer=None, init_lr_scheduler=None):
        super().__init__()
        self.model = SoftOPTModelWrapper.from_pretrained(model_name)

        # these inits should be exclusively used for loading from checkpoints
        # see load_from_custom_save for why.
        self.init_optimizer = init_optimizer
        self.init_lr_scheduler = init_lr_scheduler

        self.save_hyperparameters("model_name")

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs[0]
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss, logits = outputs[:2]

        # we care only about the last token being predicted
        pred_token_logits = logits[:, -1, :]
        pred_token = torch.argmax(pred_token_logits, dim=-1)
        labels = batch["labels"][:, -1]

        self.log("val_loss", val_loss)

        return {"loss": val_loss, "preds": pred_token, "labels": labels}

    def configure_optimizers(self):
        # thanks stack overflow!
        # https://stackoverflow.com/questions/38460918/regex-matching-a-dictionary-efficiently-in-python
        # extracting all the layers that are specified by layers_to_optimize using regex for partial matches
        regex_matches = [re.compile(".*" + pattern + ".*").match for pattern in wandb.config["layers_to_optimize"]]
        layers_to_optimize = [k for k in self.model.state_dict().keys()
                              if any(regex_match(k) for regex_match in regex_matches)]

        # configure optimizer
        optimizers_key = {"Adam": Adam, "SGD": SGD}
        if self.init_optimizer is None:
            """
            thanks forums! https://discuss.pytorch.org/t/how-to-access-to-a-layer-by-module-name/83797/8
            We cannot directly pass in the tensor output (value) from state_dict() since that is not the same reference
            as the actual layer, hence we instead look for the layer name with the regex matching, then access the 
            module by name as below.
            """
            def get_module_by_name(module: Union[torch.Tensor, nn.Module],
                                   access_string: str):
                """Retrieve a module nested in another by its access string.
                Works even when there is a Sequential in the module.
                """
                names = access_string.split(sep='.')
                return reduce(getattr, names, module)

            layers = [get_module_by_name(self.model, layer_name) for layer_name in layers_to_optimize]
            # pass in the layers into optimizer
            optimizer_type = optimizers_key[wandb.config["optimizer_type"]]
            optimizer = optimizer_type(layers, **wandb.config["optimizer_params"])
        else:
            optimizer = self.init_optimizer

        # configure learning rate scheduler
        lr_scheduler_key = {"ReduceLROnPlateau": ReduceLROnPlateau}
        if self.init_lr_scheduler is None:
            lr_scheduler_type = lr_scheduler_key[wandb.config["lr_scheduler_type"]]
            lr_scheduler = lr_scheduler_type(optimizer, **wandb.config["lr_scheduler_params"])
        else:
            lr_scheduler = self.init_lr_scheduler

        lr_scheduler_config = {"scheduler": lr_scheduler}
        lr_scheduler_config.update(wandb.config["lr_scheduler_config"])

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_config}

    @classmethod
    def load_from_custom_save(cls, model_name, path, optimizer: Optimizer = None, lr_scheduler: _LRScheduler = None):
        """
        Custom save function to load from checkpoints created by SpecificLayersCheckpoint callback.
        Unfortunately pytorch lightning locks the optimizers in place after instantiation and there is no clean way
        to change them afterwards. There are some workarounds but they all suck too:
        https://github.com/PyTorchLightning/pytorch-lightning/discussions/9354
        https://github.com/PyTorchLightning/pytorch-lightning/discussions/6131
        So the current implementation is to throw in the optimizer and lr_scheduler as optional parameters during model
        instantiation before then actually updating the model weights.
        To try different optimizers and lr_schedulers change configure_optimizers() directly.
        """
        # load the saved checkpoint
        state_dict = torch.load(path)

        # load optimizer if required
        if optimizer is not None:
            optimizer.load_state_dict(state_dict["optimizer"])

        # load lr_scheduler if required
        if lr_scheduler is not None:
            lr_scheduler.load_state_dict(state_dict["lr_scheduler"])

        # instantiate lightningmodule with pretrained model
        model = cls(model_name, optimizer, lr_scheduler)

        # load updated state dict into the model (as long as no layers are named optimizer or lr_scheduler)
        model.model.load_state_dict(state_dict, strict=False)

        return model

    """
    Note on following hooks (on_train_epoch_start and on_validation_epoch_start):
    
    Using the following code to access dataloaders: self.train_dataloader().dataset.set_epoch(self.current_epoch) 
    Results in an exception like such : pytorch_lightning.utilities.exceptions.MisconfigurationException: 
    `val_dataloader` must be implemented to be used with the Lightning Trainer 
    
    Although train_dataloader() is a valid hook, the hook is overridden only in the datamodule and we cannot reference
    that. We have to use self.trainer.train_dataloader.dataset which returns some CombinedDataset and then .datasets
    that one to get the original TorchIterableDataset.
    
    On the other hand, we can access validation dataloaders with self.trainer.val_dataloaders[0].dataset as that one is
    apparently a list and not a CombinedDataset.
    
    Pain.
    """

    def on_train_epoch_start(self) -> None:
        # reshuffle the dataset for every train epoch
        self.trainer.train_dataloader.dataset.datasets.set_epoch(self.trainer.current_epoch)

    def on_validation_epoch_start(self) -> None:
        # reshuffle the dataset for every validation epoch
        self.trainer.val_dataloaders[0].dataset.set_epoch(self.trainer.current_epoch)

##### ParaphraseOPT

In [7]:
class ParaphraseOPT(LightningModule):
    def __init__(self, model_name="facebook/opt-350m", init_optimizer=None, init_lr_scheduler=None):
        super().__init__()
        self.model = SoftOPTModelWrapper.from_pretrained(model_name)

        # these inits should be exclusively used for loading from checkpoints
        # see load_from_custom_save for why.
        self.init_optimizer = init_optimizer
        self.init_lr_scheduler = init_lr_scheduler

        self.save_hyperparameters("model_name")

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs[0]
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss, logits = outputs[:2]

        # we care only about the last token being predicted
        pred_token_logits = logits[:, -1, :]
        pred_token = torch.argmax(pred_token_logits, dim=-1)
        labels = batch["labels"][:, -1]

        self.log("val_loss", val_loss)

        return {"loss": val_loss, "preds": pred_token, "labels": labels}

    def configure_optimizers(self):
        # thanks stack overflow!
        # https://stackoverflow.com/questions/38460918/regex-matching-a-dictionary-efficiently-in-python
        # extracting all the layers that are specified by layers_to_optimize using regex for partial matches
        regex_matches = [re.compile(".*" + pattern + ".*").match for pattern in wandb.config["layers_to_optimize"]]
        layers_to_optimize = [k for k in self.model.state_dict().keys()
                              if any(regex_match(k) for regex_match in regex_matches)]

        # configure optimizer
        optimizers_key = {"Adam": Adam, "SGD": SGD}
        if self.init_optimizer is None:
            """
            thanks forums! https://discuss.pytorch.org/t/how-to-access-to-a-layer-by-module-name/83797/8
            We cannot directly pass in the tensor output (value) from state_dict() since that is not the same reference
            as the actual layer, hence we instead look for the layer name with the regex matching, then access the 
            module by name as below.
            """
            def get_module_by_name(module: Union[torch.Tensor, nn.Module],
                                   access_string: str):
                """Retrieve a module nested in another by its access string.
                Works even when there is a Sequential in the module.
                """
                names = access_string.split(sep='.')
                return reduce(getattr, names, module)

            layers = [get_module_by_name(self.model, layer_name) for layer_name in layers_to_optimize]
            # pass in the layers into optimizer
            optimizer_type = optimizers_key[wandb.config["optimizer_type"]]
            optimizer = optimizer_type(layers, **wandb.config["optimizer_params"])
        else:
            optimizer = self.init_optimizer

        # configure learning rate scheduler
        lr_scheduler_key = {"ReduceLROnPlateau": ReduceLROnPlateau}
        if self.init_lr_scheduler is None:
            lr_scheduler_type = lr_scheduler_key[wandb.config["lr_scheduler_type"]]
            lr_scheduler = lr_scheduler_type(optimizer, **wandb.config["lr_scheduler_params"])
        else:
            lr_scheduler = self.init_lr_scheduler

        lr_scheduler_config = {"scheduler": lr_scheduler}
        lr_scheduler_config.update(wandb.config["lr_scheduler_config"])

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_config}

    @classmethod
    def load_from_custom_save(cls, model_name, path, optimizer: Optimizer = None, lr_scheduler: _LRScheduler = None):
        """
        Custom save function to load from checkpoints created by SpecificLayersCheckpoint callback.
        Unfortunately pytorch lightning locks the optimizers in place after instantiation and there is no clean way
        to change them afterwards. There are some workarounds but they all suck too:
        https://github.com/PyTorchLightning/pytorch-lightning/discussions/9354
        https://github.com/PyTorchLightning/pytorch-lightning/discussions/6131
        So the current implementation is to throw in the optimizer and lr_scheduler as optional parameters during model
        instantiation before then actually updating the model weights.
        To try different optimizers and lr_schedulers change configure_optimizers() directly.
        """
        # load the saved checkpoint
        state_dict = torch.load(path)

        # load optimizer if required
        if optimizer is not None:
            optimizer.load_state_dict(state_dict["optimizer"])

        # load lr_scheduler if required
        if lr_scheduler is not None:
            lr_scheduler.load_state_dict(state_dict["lr_scheduler"])

        # instantiate lightningmodule with pretrained model
        model = cls(model_name, optimizer, lr_scheduler)

        # load updated state dict into the model (as long as no layers are named optimizer or lr_scheduler)
        model.model.load_state_dict(state_dict, strict=False)

        return model

    """
    Note on following hooks (on_train_epoch_start and on_validation_epoch_start):
    
    Using the following code to access dataloaders: self.train_dataloader().dataset.set_epoch(self.current_epoch) 
    Results in an exception like such : pytorch_lightning.utilities.exceptions.MisconfigurationException: 
    `val_dataloader` must be implemented to be used with the Lightning Trainer 
    
    Although train_dataloader() is a valid hook, the hook is overridden only in the datamodule and we cannot reference
    that. We have to use self.trainer.train_dataloader.dataset which returns some CombinedDataset and then .datasets
    that one to get the original TorchIterableDataset.
    
    On the other hand, we can access validation dataloaders with self.trainer.val_dataloaders[0].dataset as that one is
    apparently a list and not a CombinedDataset.
    
    Pain.
    """

    def on_train_epoch_start(self) -> None:
        # reshuffle the dataset for every train epoch
        self.trainer.train_dataloader.dataset.datasets.set_epoch(self.trainer.current_epoch)

    def on_validation_epoch_start(self) -> None:
        # reshuffle the dataset for every validation epoch
        self.trainer.val_dataloaders[0].dataset.set_epoch(self.trainer.current_epoch)

##### SpecificLayerCheckpoint

In [8]:
class SpecificLayersCheckpoint(Callback):
    """
    Custom saving of specific layers into a state_dict that can be loaded in using torch.load()
    Ideally, we load in the model with from_pretrained, and then use state_dict.update() to update the
    weights of the loaded model.
    """

    def __init__(self, monitor: str, dirpath: str, filename: str,
                 every_n_epochs: int, layers_to_save: List[str]):
        super().__init__()
        self.monitor = monitor
        self.dirpath = dirpath
        self.filename = filename
        self.every_n_epochs = every_n_epochs
        self.layers_to_save = layers_to_save

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        # if model should be saved this epoch (+1 since epoch count starts from 0)
        if (trainer.current_epoch + 1) % self.every_n_epochs == 0:
            # thanks stack overflow!
            # https://stackoverflow.com/questions/38460918/regex-matching-a-dictionary-efficiently-in-python
            # extracting all the layers that are specified by layers_to_save using regex for partial matches
            regex_matches = [re.compile(".*" + pattern + ".*").match for pattern in self.layers_to_save]
            save_dict = {k: v for k, v in pl_module.model.state_dict().items()
                         if any(regex_match(k) for regex_match in regex_matches)}

            # save the optimizer
            if pl_module.optimizers() is not None:
                save_dict.update({"optimizer": pl_module.optimizers().optimizer.state_dict()})

            # save the lr_scheduler
            if pl_module.lr_schedulers() is not None:
                save_dict.update({"lr_scheduler": pl_module.lr_schedulers().state_dict()})

            formatted_filename = self.filename.format(epoch=trainer.current_epoch, **trainer.callback_metrics)
            torch.save(save_dict, os.path.join(self.dirpath, formatted_filename))

In [9]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: teosh-wp19 (nlp-assignment). Use `wandb login --relogin` to force relogin


True

### SoftPrompt In Action

In [15]:
import os

import torch
import wandb
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import optuna
from optuna.trial import Trial
from optuna.integration import PyTorchLightningPruningCallback

# initialisation steps
torch.cuda.empty_cache()
AVAIL_GPUS = min(1, torch.cuda.device_count())


def objective(trial: Trial):
    # clear cache so we don't RuntimeError: CUDA out of memory. Tried to allocate 17.00 GB (GPU 0; 39.59 GiB total capacity; 37.53 GiB already allocated; 22.19 MiB free; 37.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
    torch.cuda.empty_cache()

    # initialise hyperparameter search
    trial_config = dict()

    # number of embedding tokens
    embedding_n_tokens = trial.suggest_int("embedding_n_tokens", 50, 150)
    trial_config["embedding_n_tokens"] = embedding_n_tokens
    # optimizers
    # optimizer_type = trial.suggest_categorical("optimizer_type", ["Adam", "SGD"])
    optimizer_type = "Adam"
    trial_config["optimizer_type"] = optimizer_type

    # override default params with the hyperparamters being searched for
    run = wandb.init(project="SoftTune13B",
                     name=f"optimizer_type={optimizer_type}-embedding_n_tokens={embedding_n_tokens}")
    with run:
        wandb.config.update(trial_config, allow_val_change=True)

        datamodule = DataCombiner(wandb.config["model_name"], batch_size=wandb.config["batch_size"],
                                            steps_per_epoch=wandb.config["steps_per_epoch"],
                                            datamodules=[PARABANK_Dataset, PARANMT_50M_DATASET],
                                            probabilities=[0.5, 0.5])
        datamodule.setup()

        if (wandb.config["load_from_checkpoint"] is not None) and (os.path.isfile(wandb.config["load_from_checkpoint"])):
            model = ParaphraseOPT.load_from_custom_save(wandb.config["model_name"],
                                                        wandb.config["load_from_checkpoint"])
        else:
            model = ParaphraseOPT(wandb.config["model_name"])

        checkpoint_callback = SpecificLayersCheckpoint(
            monitor="val_loss",
            dirpath=wandb.config["checkpoint_save_dir"],
            filename="soft-opt-epoch={epoch:03d}-val_loss={val_loss:.3f}" +
                     f"-optimizer_type={optimizer_type}-embedding_n_tokens={embedding_n_tokens}" + ".ckpt",
            every_n_epochs=wandb.config["checkpoint_every_n_epochs"],
            layers_to_save=wandb.config["layers_to_optimize"]
        )

        early_stopping_callback = PyTorchLightningPruningCallback(trial, monitor="val_loss")

        # create wandb logger (obviously)
        wandb_logger = WandbLogger(checkpoint_callback=False)

        print("TRAINING MODEL")
        trainer = Trainer(max_epochs=wandb.config["max_epochs"], gpus=AVAIL_GPUS,
                          check_val_every_n_epoch=wandb.config["check_val_every_n_epoch"],
                          callbacks=[checkpoint_callback, early_stopping_callback],
                          logger=wandb_logger)
        trainer.fit(model, datamodule=datamodule)

    wandb.finish()

    return trainer.callback_metrics["val_loss"].item()


if __name__ == "__main__":
    pruner: optuna.pruners.BasePruner = optuna.pruners.MedianPruner(n_warmup_steps=1000)

    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective, n_trials=2)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Using custom data configuration default-a233796b5026b737
Using custom data configuration default-12387731b9302b2b
Downloading: 100%|██████████| 2.63G/2.63G [00:32<00:00, 81.7MB/s]
c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\pytorch_lightning\loggers\wandb.py:345: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:385: LightningDeprecationWarning: The `Callback.on_epoch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_<train/validation/test>_epoch_end` instead.
  rank_zero_deprecation(


TRAINING MODEL


Using custom data configuration default-a233796b5026b737
Using custom data configuration default-12387731b9302b2b
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | SoftOPTModelWrapper | 1.3 B 
----------------------------------------------
1.3 B     Trainable params
0         Non-trainable params
1.3 B     Total params
5,264.056 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\pytorch_lightning\utilities\data.py:152: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\HADOOP\Desktop\NLP\venv\lib\site-packages\pytorch_lightning\utilities\data.py:106: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurat

...

Using custom data configuration default-a233796b5026b737
Using custom data configuration default-12387731b9302b2b
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | SoftOPTModelWrapper | 1.3 B 
----------------------------------------------
1.3 B     Trainable params
0         Non-trainable params
1.3 B     Total params
5,264.212 Total estimated model params size (MB)


Epoch 29: 100%|██████████| 500/500 [11:24:48<00:00, 82.18s/it, loss=0.348, v_num=v2ar]      


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▆▄▃▂▂▂▂▁▁▁▁▁▁▁
epoch,29
train_loss,0.36803
trainer/global_step,7499
val_loss,0.34661


[I 2022-12-26 12:59:15,912] Trial 1 finished with value: 0.34660622477531433 and parameters: {'embedding_n_tokens': 144}. Best is trial 1 with value: 0.34660622477531433.


Number of finished trials: 2
Best trial:
  Value: 0.34660622477531433
  Params: 
    embedding_n_tokens: 144


### Model Evaluation

##### BART METRIX

In [10]:
from typing import Dict, Any, List

from torch import Tensor
import torch.nn as nn
import torch

from torchmetrics import Metric
from transformers import BartTokenizer, BartForConditionalGeneration


class BartScore(Metric):
    """
    Torchmetric version of BartScore as adapted from their github
    https://github.com/neulab/BARTScore
    With lots of reference to bertscore implementation
    https://github.com/PyTorchLightning/metrics/blob/master/torchmetrics/text/bert.py#L40-L235
    Compute the score by:
    ```
    bartscore = BartScore()
    score = bartscore(['This is interesting.', 'This is a good idea.'], ['This is fun.', 'Sounds like a good idea.'])
    ```
    and it should return [-2.152808666229248, -2.948076009750366].
    """

    def __init__(self, device='cuda:0', max_length=1024, checkpoint='facebook/bart-large-cnn',
                 **kwargs: Dict[str, Any]):
        super().__init__(**kwargs)

        # Set up model
        self.device_ = device
        self.max_length = max_length
        self.tokenizer = BartTokenizer.from_pretrained(checkpoint)
        self.model = BartForConditionalGeneration.from_pretrained(checkpoint)
        self.model.eval()
        self.model.to(device)

        # Set up loss
        self.loss_fct = nn.NLLLoss(reduction='none', ignore_index=self.model.config.pad_token_id)
        self.lsm = nn.LogSoftmax(dim=1)

        # Set up metric state variables which keep track of state on each call of update
        self.add_state("src_input_ids", [], dist_reduce_fx="cat")
        self.add_state("src_attention_mask", [], dist_reduce_fx="cat")
        self.add_state("target_input_ids", [], dist_reduce_fx="cat")
        self.add_state("target_attention_mask", [], dist_reduce_fx="cat")

    def update(self, preds: List[str], target: List[str]) -> None:
        # dict of 2d list of tensors [batch_size, input_size] although input_size is not fixed
        encoded_src = self.tokenizer(preds, padding=True, return_tensors='pt')
        encoded_targets = self.tokenizer(target, padding=True, return_tensors='pt')

        # 3d list of 2d tensors, since default values of state variables can only be lists or tensors
        self.src_input_ids.append(encoded_src['input_ids'])
        self.src_attention_mask.append(encoded_src['attention_mask'])
        self.target_input_ids.append(encoded_targets['input_ids'])
        self.target_attention_mask.append(encoded_targets['attention_mask'])

    def compute(self):
        score_list = []

        src_tokens = self.src_input_ids[0].to(self.device_)
        src_mask = self.src_attention_mask[0].to(self.device_)

        tgt_tokens = self.target_input_ids[0].to(self.device_)
        tgt_mask = self.target_attention_mask[0]
        tgt_len = tgt_mask.sum(dim=1).to(self.device_)

        # while we do not use the loss computation as a result of labels being provided, the labels also cause
        # https://github.com/huggingface/transformers/blob/v4.17.0/src/transformers/models/bart/modeling_bart.py#L1320
        # the decoder input id to be shifted to the right for us, which is needed for this to work
        output = self.model(
            input_ids=src_tokens,
            attention_mask=src_mask,
            labels=tgt_tokens
        )

        # loss calculation based on original bart_score
        logits = output.logits.view(-1, self.model.config.vocab_size)
        lsm_output = self.lsm(logits)
        loss = self.loss_fct(lsm_output, tgt_tokens.view(-1))
        loss = loss.view(tgt_tokens.shape[0], -1)
        loss = loss.sum(dim=1) / tgt_len
        curr_score_list = [-x.item() for x in loss]
        score_list += curr_score_list

        return score_list


def main():
    bartscore = BartScore()
    score = bartscore(['This is interesting.', 'This is interesting.'],
                      ['This is very curious.', 'This is incredibly strange.'])
    print(score)


if __name__ == "__main__":
    main()

[-2.2617740631103516, -2.824507474899292]


##### Generate prediction sample

In [18]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df=pd.read_pickle(r"C:\Users\HADOOP\Desktop\NLP\EVALUATION\Preds\soft-samples=500.pkl")

In [20]:
df.head(500)

,preds,src
0,,and we shall call him john .
1,,"i musthave loved you for years , only ... ... i was such a fool , i did n't know it ."
2,,allison : terry knows no-one said anything ?
3,,You must be older than I thought.
4,,"these guys manipulate markets , and they create insurgencies to start wars ."
...,...,...
495,,"where a reference is made to this paragraph , article 5 of regulation ( eu ) no 182/2011 shall apply ."
496,,Athletics at the 2012 Summer Olympics
497,,there was n't enough time .
498,,United States at the 1956 Winter Olympics


In [22]:
wandb.init("See embedding")
def reconstruct_tokens(model):
    """
    Find the nearest tokens in the embedding space.
    https://stackoverflow.com/questions/64523788/how-to-invert-a-pytorch-embedding
    """
    embeddings = model.model.soft_embedding.wte
    learned_embedding = model.model.soft_embedding.learned_embedding

    reconstructed = list()
    for i in learned_embedding:
        distance = torch.norm(embeddings.weight.detach() - i, dim=1)
        nearest = torch.argmin(distance)
        reconstructed.append(nearest.item())

    return reconstructed

model = ParaphraseOPT.load_from_custom_save("facebook/opt-1.3b",
r"C:\Users\HADOOP\Desktop\NLP\training_checkpoints\07-06-2022-optimize\soft-opt-epoch=029-val_loss=0.347-optimizer_type=Adam-embedding_n_tokens=144.ckpt")
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-1.3b")
tokens = reconstruct_tokens(model)
tokenizer.batch_decode(tokens)

['mur',
 'ÛÛ',
 ' Mang',
 'sha',
 ' Silver',
 'ze',
 'HI',
 'aby',
 ' Nanto',
 ' forward',
 'é',
 'ub',
 'g',
 'ood',
 'ty',
 'acht',
 ';;;;',
 ' special',
 'cool',
 ' �',
 ' RE',
 'ial',
 'town',
 'ces',
 '′',
 'se',
 ' board',
 '////////////////////////////////',
 ' realistic',
 ' Licensed',
 ' prescribing',
 ' Rome',
 'lig',
 ' proportion',
 ' open',
 ' endif',
 'burgh',
 ' awa',
 'Supporters',
 ' fro',
 ' Today',
 ' Public',
 ' Prom',
 ' Marshall',
 ' destroyer',
 ' m',
 'eries',
 '.',
 ' Sig',
 ' Par',
 'itching',
 ' need',
 'pers',
 ' U',
 ' missed',
 ' gathering',
 'roy',
 'Reviewer',
 ' seriously',
 'igh',
 'age',
 ' Berg',
 'TA',
 'enge',
 ' pick',
 ' mileage',
 ' he',
 ' play',
 ' sufficient',
 'or',
 ' filmed',
 'ne',
 ' key',
 'stone',
 'ova',
 'elt',
 ' 1000',
 ' sol',
 ' Que',
 ' Blues',
 'everal',
 ' fer',
 'hi',
 'iru',
 'Grand',
 ' vide',
 'ysc',
 ' Constitutional',
 'handed',
 ' Talk',
 ' Wave',
 'cia',
 ' Prin',
 ' Brom',
 'land',
 ' swing',
 'BS',
 ' kind',
 'ter',


In [30]:

model = model.to("cuda")

model_prompt_key = {'OPT1.3B Prompt Fine Tuned': lambda x: x + "</s>",
                    'OPT1.3B Base Model': lambda x: x}



def tokenize(model_type: str, prompt: str):
    soft_prompt = model_prompt_key[model_type](prompt)
    encoded_inputs = tokenizer(soft_prompt, return_tensors="pt")
    return encoded_inputs


tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-350m")


def predict(model_type: str, prompt: str, max_len: int):
    inputs = tokenize(model_type, prompt).to("cuda")
    if model_type == "OPT1.3B Base Model":
        outputs = model.generate(inputs.input_ids, max_length=max_len, use_cache=False)
    else:
        outputs = model.model.generate(inputs.input_ids, max_length=max_len, use_cache=False)
    outputs = outputs[:, inputs['input_ids'].size(dim=-1):]
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return decoded



In [47]:
import nltk
import string
from nltk.tokenize import word_tokenize

tuparr=[]
with open(r'C:\Users\HADOOP\Desktop\NLP\para-nmt-5m-processed\para-nmt-5m-processed.txt', encoding="utf8") as f:
    for line in f:
        source = line.split("\t")[0]
        prediction = predict('OPT1.3B Prompt Fine Tuned', source, 45)
        no_punct1 = source.translate(str.maketrans('', '', string.punctuation))
        no_punct2 = prediction.translate(str.maketrans('', '', string.punctuation))

        if(source != prediction):
            if (len(word_tokenize(no_punct1)) != len(word_tokenize(no_punct2))):
                tuparr.append((source, prediction))
                print(len(tuparr))
        if len(tuparr) == 500:
            break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


Input length of input_ids is 52, but `max_length` is set to 45. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


Input length of input_ids is 81, but `max_length` is set to 45. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490


In [48]:
tuparr

[("so , unless that 's gon na be feasible , then ...",
  "unless that's gon'be possible, then..."),
 ("by now , singh 's probably been arrested .",
  "now, singh's probably been arrested."),
 ('“ why not ?', '“ why not? ”'),
 ("an old man 's mistake … '", "an old man's mistake."),
 ('he reached for a microphone and i could hear his voice booming faintly throughout the ship .',
  'he reached for a microphone and i could hear his voice booming through the ship.'),
 ('he loved that little man , by the way .', 'he loved that little man.'),
 ("i 've been very , very lucky .", "i've been very lucky."),
 ("well , then , i 'm kind of gon na rub this dinner in her face next time i see her .",
  "well, i'm going to rub this dinner in her face next time i see her."),
 ("it 's like , gross , yet oddly delicious .",
  "it's like, gross, yet oddly delicious."),
 ("not as much as i 'd like", "not as much as i 'd like to see."),
 ("there 's no doubt in my mind that this is the right group of men but w

In [49]:
df = pd.DataFrame(tuparr, columns=["source","target"])

In [50]:
df.to_csv("500prediction.csv")

In [52]:
pd.set_option('display.max_colwidth', None)

df.head(40
)

,source,target
0,"so , unless that 's gon na be feasible , then ...","unless that's gon'be possible, then..."
1,"by now , singh 's probably been arrested .","now, singh's probably been arrested."
2,“ why not ?,“ why not? ”
3,an old man 's mistake … ',an old man's mistake.
4,he reached for a microphone and i could hear his voice booming faintly throughout the ship .,he reached for a microphone and i could hear his voice booming through the ship.
5,"he loved that little man , by the way .",he loved that little man.
6,"i 've been very , very lucky .",i've been very lucky.
7,"well , then , i 'm kind of gon na rub this dinner in her face next time i see her .","well, i'm going to rub this dinner in her face next time i see her."
8,"it 's like , gross , yet oddly delicious .","it's like, gross, yet oddly delicious."
9,not as much as i 'd like,not as much as i 'd like to see.


##### Define Metrix

In [73]:
from torchmetrics.text.bleu import BLEUScore
from torchmetrics.text.rouge import ROUGEScore

def benchmark_pairs(filepath):
    print("Loading for predictions.")
    df = pd.read_csv(filepath)[["source","target"]]

    # init metrics
    bart = BartScore()
    rouge = ROUGEScore()
    bleu = BLEUScore()

    # apply the metrics on the source and target sentence
    def score(row):
        src, target = row
        bartscore = bart([src], [target])[0]
        bleuscore = bleu([src], [[target]]).item()
        rougescore = {k: v.item() for k, v in rouge(src, target).items()}
        series = pd.Series([src, target, bartscore, bleuscore], index=["src", "target", "bartscore", "bleuscore"])
        return pd.concat([series, pd.Series(rougescore)])

    # apply score function along each row
    print("Scoring sequence pairs.")
    df = df.apply(score, axis=1)
    print(df)
    df.to_csv("benchmark_result.csv")

##### Benchmark in action

In [74]:
benchmark_pairs(r"C:\Users\HADOOP\Desktop\NLP\500prediction.csv")

Loading for predictions.
Scoring sequence pairs.
                                                                                                      src  \
0                                                       so , unless that 's gon na be feasible , then ...   
1                                                              by now , singh 's probably been arrested .   
2                                                                                             “ why not ?   
3                                                                               an old man 's mistake … '   
4            he reached for a microphone and i could hear his voice booming faintly throughout the ship .   
..                                                                                                    ...   
495                                              no , do n't worry , he 's not gon na live here forever .   
496                                               does n't it tear your guts to

In [77]:
df = pd.read_csv(r"benchmark_result.csv").head()

In [78]:
df.mean()

C:\Users\HADOOP\AppData\Local\Temp\ipykernel_21468\3698961737.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.mean()


Unnamed: 0             2.000000
bartscore             -1.766973
bleuscore              0.137793
rouge1_fmeasure        0.915261
rouge1_precision       0.879762
rouge1_recall          0.958095
rouge2_fmeasure        0.833050
rouge2_precision       0.801667
rouge2_recall          0.871429
rougeL_fmeasure        0.915261
rougeL_precision       0.879762
rougeL_recall          0.958095
rougeLsum_fmeasure     0.915261
rougeLsum_precision    0.879762
rougeLsum_recall       0.958095
dtype: float64